# Задача 1 Права администратора

Довольно часто в веб-разработке приходится иметь дело с проблемой ограничения доступа. Некоторые авторизованные пользователи должны иметь доступ к ресурсу, другие же нет. Для решения этой проблемы используются роли: администратор, менеджер, пользователь и т.д. и т.п.

Легче всего решить эту проблему при помощи декоратора ``role_required(role: str)``, который будет разрешать или запрещать определенные действия, выполняемые над ресурсом, в нашем примере это будет функция ``secret_resource() -> str``.

Исходные условия:

- Определена функция *secret_resource() -> str*, которая должна быть
доступна только пользователям с ролью *'admin'*.

- Роль очередного пользователя записана в глобальной переменной *user_role*.

Порядок выполнения:

Напишите код декоратора ``role_required(role: str)``, который в случае, если пользователь является админом предоставляет доступ к функции
*secret_resource*, а иначе возвращает строку **'Permission denied'**.

- Вход 1: *admin*

- Выход 1: **Permission accepted**

- Вход 2: *manager*

- Выход 2: **Permission denied**

In [1]:
import functools

USER_ROLE = {"role": None}

def role_required(role):

    """
    Декоратор ограничивает доступ к функции в зависимости от роли пользователя.
    """
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if USER_ROLE == role:
                return func(*args, **kwargs)
            return "Permission denied"
        return wrapper
    return decorator

@role_required("admin")
def secret_resource():
    """
    Функция дает доступ для пользователя admin
    """
    return "Permission accepted"


In [2]:
USER_ROLE = "admin"
print(f"{USER_ROLE}: {secret_resource()}")

USER_ROLE = "manager"
print(f"{USER_ROLE}: {secret_resource()}")

USER_ROLE = "simple_user"
print(f"{USER_ROLE}: {secret_resource()}")

admin: Permission accepted
manager: Permission denied
simple_user: Permission denied


# Задача 2 Кэширование

Кэширование помогает сохранять результат запроса (например, из БД) и выдавать его сразу, не совершая новых запросов в БД. Важно определить политику кэширования - время жизни данных в секундах или количество раз, сколько их можно запросить (после чего они будут стёрты из кэша). Мы будем использовать вторую политику для простоты.

В этом задании вам придётся реализовать декоратор, который будет принимать несколько аргументов. Так, нужно написать декоратор ``cache(db: str)``, который принимает в качестве параметра *db* - название базы данных, где будет кэшироваться информация. Затем подумайте, как можно передать второй параметр - *expiration*, количество раз, когда данные будут браться из кэша, а затем будут стёрты.

При первом запросе необходимо кэшировать результат и возвращать строку вида:

Info about: <thing> from <db>, now cached with expire=<expire_time>

Где:

- *thing* - параметр функции get_info (возвращающая информацию о предмете);
- *db* - название БД, где будет кэшироваться информация;
- *expire_time* - количество раз, когда данные будут браться из кэша, а затем будут стёрты.

При последующих запросах необходимо выдавать следующие строки:

``Info about: <thing> cached in <db>, expire=<expiration_time-1>``

``Info about: <thing>cached in <db>, expire=<expiration_time-2>``

``Info about: <thing>cached in <db>, expire=<expiration_time-3>``

Вплоть до:

``Info about: <thing>cached in <db>, expire=0``

Исходные условия:

- Определена функция *get_info(thing: str) -> str*, которая возвращает информацию о предмете *thing*.

In [3]:
import functools

CACHE_DICT = {}

def get_info(thing):
  """
  Функция возвращает информацию о thing
  """
  return f"{thing} info"

def cache(db, expiration_time):
    """
    Декоратор добавляет информацию о thing и об expiration_time - количестве раз,
    когда данные будут браться из кэша, а затем будут стёрты - в базу данных db.

    """

    def decorator(func):

        @functools.wraps(func)
        def wrapper(thing):
            cache_key = (f"db={db},expiration_time={expiration_time},thing={thing}")
            if cache_key not in CACHE_DICT:
                CACHE_DICT[cache_key] = [get_info(thing), expiration_time]
                print(f"Info about: {thing} from {db}, now cached with expire={CACHE_DICT[cache_key][1]}")
                CACHE_DICT[cache_key][1] -= 1
                return CACHE_DICT[cache_key][0]
            if CACHE_DICT[cache_key][1] > 0:
                print(f"Info about: {thing} cached in {db}, expire={CACHE_DICT[cache_key][1]}")
                CACHE_DICT[cache_key][1] -= 1
                return CACHE_DICT[cache_key][0]
            print(f"Info about: {thing} cached in {db}, expire={CACHE_DICT[cache_key][1]}")
            del CACHE_DICT[cache_key] # очистка кеша

        return wrapper
    return decorator

In [4]:
@cache("postgresql", 5)
def get_info_1(thing):
    return get_info(thing)

@cache("sqlite", 3)
def get_info_2(thing):
    return get_info(thing)

In [5]:
for i in range(7):
    get_info_1("bike_store")

for i in range(5):
    get_info_2("bike_store")

Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store cached in postgresql, expire=4
Info about: bike_store cached in postgresql, expire=3
Info about: bike_store cached in postgresql, expire=2
Info about: bike_store cached in postgresql, expire=1
Info about: bike_store cached in postgresql, expire=0
Info about: bike_store from postgresql, now cached with expire=5
Info about: bike_store from sqlite, now cached with expire=3
Info about: bike_store cached in sqlite, expire=2
Info about: bike_store cached in sqlite, expire=1
Info about: bike_store cached in sqlite, expire=0
Info about: bike_store from sqlite, now cached with expire=3


In [6]:
for i in range(7):
    get_info_1("users")

for i in range(5):
    get_info_2("users")

Info about: users from postgresql, now cached with expire=5
Info about: users cached in postgresql, expire=4
Info about: users cached in postgresql, expire=3
Info about: users cached in postgresql, expire=2
Info about: users cached in postgresql, expire=1
Info about: users cached in postgresql, expire=0
Info about: users from postgresql, now cached with expire=5
Info about: users from sqlite, now cached with expire=3
Info about: users cached in sqlite, expire=2
Info about: users cached in sqlite, expire=1
Info about: users cached in sqlite, expire=0
Info about: users from sqlite, now cached with expire=3


# Задача 3 Контекстный менеджер safe_write

Реализуйте контекстный менеджер ``safe_write``, который принимает один аргумент: filename — имя файла.

Контекстный менеджер должен позволять записывать информацию в файл с именем *filename*. Причем если во время записи в файл было возбуждено какое-либо исключение, контекстный менеджер должен поглотить его, отменить все выполненные ранее записи в файл, если они были, вернуть файл в исходное состояние и проинформировать о возбужденном исключении выводом следующего текста:

*Во время записи в файл было возбуждено исключение <тип исключения>*

Дополнительная проверка данных на корректность не требуется. Гарантируется, что реализованный контекстный менеджер используется только с корректными данными.

In [7]:
from contextlib import contextmanager

@contextmanager
def safe_write(file_name,  mode="w", encoding="utf-8"):
    """
    Контекстный менеджер записывает в файл текст. Последнюю перед ошибкой запись записываем
    в cache и используем ее после ошибки

    """
    # Кэш
    try:
        with open(file_name, mode="r", encoding=encoding) as file:
            cache = file.read()
    except:
        pass

    # Функция
    try:
        with open(file_name, mode=mode, encoding=encoding) as file:
            yield file
    except Exception as e:
        print(f"Во время записи в файл было возбуждено исключение {type(e).__name__}")
        with open(file_name, mode=mode, encoding=encoding) as file:
            file.write(cache)

In [8]:
with safe_write('test_1.txt') as file:
    file.write('Я знаю, что ничего не знаю, но другие не знают и этого.')
with open('test_1.txt', encoding="utf-8") as file:
    print(file.read())

Я знаю, что ничего не знаю, но другие не знают и этого.


In [9]:
with safe_write('test_2.txt') as file:
    file.write('Я знаю, что ничего не знаю, но другие не знают и этого. \n')

with safe_write('test_2.txt') as file:
    print('Если ты будешь любознательным, то будешь много знающим.',
          file=file,
          flush=True
          )
    raise ValueError

with open('test_2.txt', encoding="utf-8") as file:
    print(file.read())

Во время записи в файл было возбуждено исключение ValueError
Я знаю, что ничего не знаю, но другие не знают и этого. 

